In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from skimage import io
from skimage.exposure import rescale_intensity
import cv2
from ultralytics import YOLO

In [ ]:
structure = '10N_R'
atlas_path = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/atlasV8'
volume_path = os.path.join(atlas_path, 'structure', f'{structure}.npy')
volume = np.load(volume_path)
volume.shape

In [ ]:
model = YOLO("yolov8x-seg.pt")

In [ ]:
filepath = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/masks/normalized/DK41.037.tif'
#img[img == 255] = 0
img.dtype

In [ ]:
#img = io.imread(filepath)
# {object_class_id} {x_center} {y_center} {width} {height}
bigtissue = 1000
smalltissue = 1001
barcode = 1003
junk = 1004

img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
ret, thresh = cv2.threshold(img, 10, 255, cv2.THRESH_BINARY)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
boxes = []
imgw = img.shape[1]
imgh = img.shape[0]
big_contour = max(contours, key=cv2.contourArea)
bigarea = cv2.contourArea(big_contour)
bx,by,bw,bh = cv2.boundingRect(big_contour)
boxes.append([bigtissue, ((bx+bw)/2)/imgw, ((by+bh)/2)/imgh, bw/imgw, bh/imgh])
for contour in contours:
    x,y,w,h = cv2.boundingRect(contour)
    area = cv2.contourArea(contour)
    
    if area > 25 and area < bigarea:
        x_center = ((x+w)/2)/imgw
        y_center = ((y+h)/2)/imgh
        width = w/imgw
        height = h/imgh
        if x > imgw * 0.9:
            id = barcode
        else:
            id = smalltissue
        cv2.rectangle(img,(x,y),(x+w,y+h),255, 1)
        boxes.append([id, x_center, y_center, width, height])
boxes

In [ ]:
results = model.predict(filepath)

In [ ]:
result = results[0]
len(result.boxes)
box = result.boxes

In [ ]:
print("Object type:", box.cls)
print("Coordinates:", box.xyxy)
print("Probability:", box.conf)

In [ ]:
cords = box.xyxy[0].tolist()
class_id = box.cls[0].item()
class_id = result.names[box.cls[0].item()]
conf = round(box.conf[0].item(), 2)
print("Object type:", class_id)
print("Coordinates:", cords)
print("Probability:", conf)

In [ ]:
plt.imshow(img, cmap='gray')

In [ ]:
smooth_img = smooth_image(img)
plt.imshow(smooth_img, cmap='gray')

In [ ]:
%%time
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

cmap = plt.get_cmap("viridis")
norm= plt.Normalize(volume.min(), volume.max())
ax.voxels(np.ones_like(volume), facecolors=cmap(norm(volume)), edgecolor="black")
#ax.voxels(volume)
plt.show()